In [1]:
import os
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core.compute import ComputeTarget
from azureml.contrib.core.compute.itpcompute import ItpCompute
from azureml.core.container_registry import ContainerRegistry
from azureml.core.runconfig import TensorflowConfiguration
from azureml.train.dnn import TensorFlow
import azureml.core
print(f"The azureml-sdk version is {azureml.core.VERSION}")

The azureml-sdk version is 1.6.0


In [2]:
container_registry = ContainerRegistry()
container_registry.address = 'philly2aml.azurecr.io'
container_registry.username = 'philly2aml'
container_registry.password = 'CiuybPECAy1UOvXGyC91ujZm3FU6V5d/'
script_folder = './tf-mnist'

In [3]:
ws = Workspace.from_config()

WARNING - Warning: Falling back to use azure cli login credentials.
If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [4]:
ws

Workspace.create(name='cmaksphillywestcentralus', subscription_id='06237282-1ddd-4677-9878-5bc19cc0001a', resource_group='ITPTestingWorkers')

In [5]:
compute_name = 'cmakstestgpu2'
cmaks_compute = ComputeTarget(workspace=ws, name=compute_name)

In [6]:
exp = Experiment(workspace=ws, name='tf-mnist-prebuild')

In [8]:
# Dockerfile for tf-dist-mnist-test
#
# ```
# FROM tensorflow/tensorflow:1.15.0-gpu-py3
# RUN pip install azureml.core
#
# ```

distributed_training = TensorflowConfiguration()
distributed_training.parameter_server_count = 1
distributed_training.worker_count = 2
# distributed_training.process_count_per_node = 2
tf_est = TensorFlow(source_directory=script_folder,
                    entry_script='tf_mnist_dist.py',
                    compute_target=cmaks_compute,
                    image_registry_details=container_registry,
                    custom_docker_image="tensorflow-test/tf-dist-mnist-test:gpu",
                    user_managed=True,
                    node_count=3,
                    process_count_per_node=2,
#                     distributed_training=distributed_training,
                    use_gpu=True)

WARNING - 'process_count_per_node' parameter will be deprecated. Please use it as part of 'distributed_training' parameter.
WARNING - 'process_count_per_node' parameter will be used only in combination with 'distributed_backend' parameter. As both these parameters are to be deprecated, please use 'distributed_training' parameter and set 'process_count_per_node' on that object.


TrainingException: TrainingException:
	Message: 'distributed_backend' or 'distributed_training' must be specified when node_count > 1 or process_count_per_node > 1.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "'distributed_backend' or 'distributed_training' must be specified when node_count > 1 or process_count_per_node > 1."
    }
}

In [ ]:
run = exp.submit(tf_est)

In [ ]:
# run.wait_for_completion(show_output=True)

In [ ]:
# for i in range(50):
#     run = exp.submit(tf_est)